#  Terminology Project - M2 TAL - 24/11/2020

## Axel Didier - Pierre Goncalves - M2 TAL

### REQUIREMENTS

In [65]:
#Packages : os, spacy, flair

import sys

#!{sys.executable} -m pip install flair
#!{sys.executable} -m pip install spacy


## Rule-based identification system

In [66]:
import os
import re
import spacy

nlp = spacy.load("en_core_web_sm")

### Create a .txt lexicon with a .tsv as input

In [67]:
def clean_term():
    f = open("TermsTSV/MOBA-en.tsv", "r")
    raw = f.read()
    
    #get term raw of the tsv file
    raw_terms = re.findall('\:\s(.*[a-z])', raw)

    result = []
    #loop on every terms
    for item in raw_terms:
        lemmatized_term = ''
        #loop on every token of the term
        doc = nlp(item)
        for token in doc:
            lemmatized_term += token.lemma_ + ' '
            
        print(lemmatized_term[:-1]) #-1 to remove the last white space
        
        #to avoid doublon
        if lemmatized_term not in result:
            result.append(lemmatized_term)
            
    #write all terms in a file
    with open('MOBA-en-lexicon.txt', 'w') as f:
        for item in result:
            f.write(item+"\n")
            
clean_term()

moba
avatar
game avatar
moba game
toxic player
toxic and typical player
toxic and normal player
typical and toxic player
player skill
player base skill
player skill decomposition
player skill composition
player skill evaluation
player skill formation
player skill rating
player skill analysis
team composition
dota
dota
creeps
team formation
league of legend
online game
hot
dot
typical player
gameplay
enemy team
team fight
defense
in - game
riot game
blue team
red and blue team
enemy hero
tactical analysis
late game
player behavior
prize
champion attribute
opposite team
play style
user report
red team
red and blue team
experienced role
inexperienced role
minion
mana
creep
easy mode
red side
blue side
normal player
novice player
lol
lol player
game match
enemy base
noob
mage
matchmaking
skill rating
win rate
low winning rate
enemy tower
hard mode
high skill player
target selection
gamer
opponent team
team role
amount of gold
skillful
gameplay style
neutral creep
rating system
elo rating s

### Get functions (lexicon, article.txt, list of terms in the lexicon)

In [68]:
def get_lexicon():
    f = open("MOBA-en-lexicon.txt", "r")
    lexicon = f.read().splitlines() #use splitlines to not take '\n' char
    #return a list of splited list (to ease the further process)
    return [item.split() for item in lexicon]

def get_doc(file_name):
    f = open(file_name, "r", encoding="utf8")
    text = f.read()
    print(file_name, '|| Text length: ',len(text))
    return text

def get_possible_term(lexicon, word):
    possible_term = []
    for item in lexicon:
        if word == item[0]:
            possible_term.append(item)
    return possible_term

lexicon = get_lexicon()

In [69]:
# Main function to create an IOB annotated .txt file
# Manage one joker word / don't manage syntactic variants
def annotate_doc(txt, f):
    
    txt_pos=0
    while txt_pos < len(txt):#in range(0, len(txt)):
        word = str(txt[txt_pos].lemma_)
        #print(">>", word)
        
        possible_terms = get_possible_term(lexicon, word)
        match = possible_terms
        #print("Possible: ",match)
        
        # If there is possible_terms
        if possible_terms is not None:
            
            to_remove = []
            joker = True
            joker_pos = None
            joker_word = None
            # loop on them
            for term in possible_terms:
                #print(">", term)
                
                # Process for each term word
                for term_pos in range(0, len(term)):
                
                    #check if we dont go out the text length
                    if txt_pos+term_pos < len(txt):
                        #check if word term are not the same of the text word
                        if term[term_pos] != str(txt[txt_pos+term_pos].lemma_):
                            #if not the same but the next is the same, means that still have to process (rule)
                            if term[term_pos] == str(txt[txt_pos+term_pos + 1].lemma_) and joker is True:
                                #joker to False because we can only do that one time
                                joker = False
                                joker_pos = term_pos
                                joker_word = str(txt[txt_pos+term_pos])
                                break
                            else:
                                to_remove.append(term)
                                break
                    else:
                        to_remove.append(term)
                        break
            
            #remove all the terms that didn't match
            for t in to_remove:
                match.remove(t)
                
            # if there is a match
            if len(match)>0:
                #take the longuest
                real_match = max(match, key=len)
                #print("Real Match: ", real_match)
                
                # and jump to the next word
                if joker is False: 
                    f.write(str(txt[txt_pos]) + " B\n")
                    #print("Write: " + str(txt[txt_pos]) + " B\n")
                    for i in range(1, len(real_match)+1):
                        if joker_pos == i:
                            f.write(joker_word + " O\n")
                            #print("Write: " + joker_word + " O\n")
                        else: 
                            f.write(str(txt[txt_pos+i]) + " I\n")
                            #print("Write: " + str(txt[txt_pos+i]) + " I\n")
                    txt_pos += len(real_match)+1
                        
                else: 
                    
                    f.write(str(txt[txt_pos]) + " B\n")
                    #print("Write: " + str(txt[txt_pos]) + " B\n")
                    for i in range(1, len(real_match)):
                        f.write(str(txt[txt_pos+i]) + " I\n")
                        #print("Write: " + str(txt[txt_pos+i]) + " I\n")
                    txt_pos += len(real_match)
            else:
                if word is not "\n":
                    f.write(str(txt[txt_pos])+" O\n")
                    #print("Write: " + str(txt[txt_pos])+" O\n")
                txt_pos += 1
                if word is ".": f.write("\n")

### Creation of the annotated Corpus with IOB (.txt)
#### COMMENT the last line if you don't want to run it (already train.txt already exist) 

In [70]:
# Create TRAIN.TXT file
def train():
    f = open('Corpus/train.txt', 'w', encoding="utf8")
    for filename in os.listdir("Articles/txt/")[0:20]:
        if filename.endswith(".txt"):
            print(filename)
            doc = nlp(get_doc("Articles/txt/"+filename))
            annotate_doc(doc, f)
    f.close()

# comment/uncomment if you want
train()

doc_01.txt
Articles/txt/doc_01.txt || Text length:  17603
doc_02.txt
Articles/txt/doc_02.txt || Text length:  29695
doc_03.txt
Articles/txt/doc_03.txt || Text length:  70327
doc_04.txt
Articles/txt/doc_04.txt || Text length:  22563
doc_05.txt
Articles/txt/doc_05.txt || Text length:  23758
doc_06.txt
Articles/txt/doc_06.txt || Text length:  32981
doc_07.txt
Articles/txt/doc_07.txt || Text length:  18915
doc_08.txt
Articles/txt/doc_08.txt || Text length:  45445
doc_09.txt
Articles/txt/doc_09.txt || Text length:  42509
doc_10.txt
Articles/txt/doc_10.txt || Text length:  43999
doc_11.txt
Articles/txt/doc_11.txt || Text length:  54214
doc_12.txt
Articles/txt/doc_12.txt || Text length:  78258
doc_13.txt
Articles/txt/doc_13.txt || Text length:  27403
doc_14.txt
Articles/txt/doc_14.txt || Text length:  25424
doc_15.txt
Articles/txt/doc_15.txt || Text length:  38869
doc_16.txt
Articles/txt/doc_16.txt || Text length:  38604
doc_17.txt
Articles/txt/doc_17.txt || Text length:  63251
doc_18.txt
Art

In [71]:
# Create TEST.TXT file
def test():
    f = open('Corpus/test.txt', 'w', encoding="utf8")
    for filename in os.listdir("Articles/txt/")[20:25]:
        if filename.endswith(".txt"):
            print(filename)
            doc = nlp(get_doc("Articles/txt/"+filename))
            annotate_doc(doc, f)
    f.close()

# comment/uncomment if you want 
test()

doc_21.txt
Articles/txt/doc_21.txt || Text length:  43704
doc_22.txt
Articles/txt/doc_22.txt || Text length:  20314
doc_23.txt
Articles/txt/doc_23.txt || Text length:  18202
doc_24.txt
Articles/txt/doc_24.txt || Text length:  42682
doc_25.txt
Articles/txt/doc_25.txt || Text length:  51958


In [72]:
# Create DEV.TXT file
def dev():
    f = open('Corpus/dev.txt', 'w', encoding="utf8")
    for filename in os.listdir("Articles/txt/")[25:30]:
        if filename.endswith(".txt"):
            print(filename)
            doc = nlp(get_doc("Articles/txt/"+filename))
            annotate_doc(doc, f)
    f.close()

# comment/uncomment if you want 
dev()

doc_26.txt
Articles/txt/doc_26.txt || Text length:  60221
doc_27.txt
Articles/txt/doc_27.txt || Text length:  51510
doc_28.txt
Articles/txt/doc_28.txt || Text length:  92987
doc_29.txt
Articles/txt/doc_29.txt || Text length:  55843
doc_30.txt
Articles/txt/doc_30.txt || Text length:  19363


## Sequence Tagger 

### Reading Your Own Sequence Labeling Dataset using FLAIR

In [73]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# define columns
columns = {0: 'text', 1: 'iob'}

# this is the folder in which train, test and dev files reside
data_folder = './Corpus/'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.txt',
                              test_file='test.txt',
                              dev_file='dev.txt')

2020-11-19 16:53:28,342 Reading data from Corpus
2020-11-19 16:53:28,343 Train: Corpus\train.txt
2020-11-19 16:53:28,343 Dev: Corpus\dev.txt
2020-11-19 16:53:28,344 Test: Corpus\test.txt


In [74]:
len(corpus.train)

8001

In [75]:
print(corpus.train[4].to_tagged_string('iob'))

An Analysis of DOTA2 Using Game Refinement Measure .


### Training a Sequence Labeling Model

In [76]:
from flair.data import Corpus
from flair.datasets import UD_ENGLISH
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings

# 1. get the corpus
print(corpus)

# 2. what tag do we want to predict?
tag_type = 'iob'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary)

# 4. initialize embeddings
embedding_types = [

    WordEmbeddings('glove'),

    # comment in this line to use character embeddings
    # CharacterEmbeddings(),

    # comment in these lines to use flair embeddings
    # FlairEmbeddings('news-forward'),
    # FlairEmbeddings('news-backward'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

# 7. start training
trainer.train('resources/taggers/example-pos',
              embeddings_storage_mode='gpu',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=10)

Corpus: 8001 train + 2092 dev + 1577 test sentences
Dictionary with 6 tags: <unk>, O, B, I, <START>, <STOP>
2020-11-19 16:53:32,363 ----------------------------------------------------------------------------------------------------
2020-11-19 16:53:32,364 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('glove')
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=6, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2020-11-19 16:53:32,364 ----------------------------------------------------------------------------------------------------
2020-11-19 16:53:32,365 Corpus: "Corpus: 8001 train + 2092 dev + 1577 test sentences"
2020-11-19 16:53:32,366 ----------------------------------------------------------------

2020-11-19 16:55:53,209 epoch 5 - iter 75/251 - loss 3.98351756 - samples/sec: 284.06 - lr: 0.100000
2020-11-19 16:55:55,742 epoch 5 - iter 100/251 - loss 3.90487113 - samples/sec: 315.93 - lr: 0.100000
2020-11-19 16:55:58,125 epoch 5 - iter 125/251 - loss 3.88102746 - samples/sec: 335.94 - lr: 0.100000
2020-11-19 16:56:00,799 epoch 5 - iter 150/251 - loss 3.84390698 - samples/sec: 299.31 - lr: 0.100000
2020-11-19 16:56:03,531 epoch 5 - iter 175/251 - loss 3.85582840 - samples/sec: 292.93 - lr: 0.100000
2020-11-19 16:56:06,376 epoch 5 - iter 200/251 - loss 3.89857929 - samples/sec: 281.35 - lr: 0.100000
2020-11-19 16:56:08,962 epoch 5 - iter 225/251 - loss 3.91408826 - samples/sec: 309.41 - lr: 0.100000
2020-11-19 16:56:11,590 epoch 5 - iter 250/251 - loss 3.89550693 - samples/sec: 304.54 - lr: 0.100000
2020-11-19 16:56:11,693 ----------------------------------------------------------------------------------------------------
2020-11-19 16:56:11,694 EPOCH 5 done: loss 3.9499 - lr 0.100

2020-11-19 16:58:56,542 EPOCH 10 done: loss 3.4684 - lr 0.1000000
2020-11-19 16:59:00,091 DEV : loss 3.035252094268799 - score 0.9595
2020-11-19 16:59:00,104 BAD EPOCHS (no improvement): 1
2020-11-19 16:59:02,583 ----------------------------------------------------------------------------------------------------
2020-11-19 16:59:02,583 Testing using best model ...
2020-11-19 16:59:02,584 loading file resources\taggers\example-pos\best-model.pt
2020-11-19 16:59:06,262 	0.9572
2020-11-19 16:59:06,262 
Results:
- F-score (micro): 0.9572
- F-score (macro): 0.6935
- Accuracy (incl. no class): 0.9572

By class:
              precision    recall  f1-score   support

           O     0.9673    0.9908    0.9789     32082
           B     0.8189    0.5969    0.6905      2235
           I     0.5483    0.3287    0.4110       432

    accuracy                         0.9572     34749
   macro avg     0.7782    0.6388    0.6935     34749
weighted avg     0.9526    0.9572    0.9533     34749

2020-1

{'test_score': 0.9572,
 'dev_score_history': [0.9346,
  0.949,
  0.951,
  0.9502,
  0.9568,
  0.9564,
  0.9593,
  0.9599,
  0.9609,
  0.9595],
 'train_loss_history': [5.588340487613146,
  4.5136235681663,
  4.234262478779037,
  4.048778550083418,
  3.9499369232778054,
  3.777298006878431,
  3.6407367171519307,
  3.589650133691461,
  3.5087066048169993,
  3.46840430635855],
 'dev_loss_history': [5.412528991699219,
  3.884171962738037,
  3.7232940196990967,
  3.687502145767212,
  3.0893001556396484,
  3.136033296585083,
  2.9495975971221924,
  2.8404431343078613,
  2.7531497478485107,
  3.035252094268799]}

### Apply your two identification systems on a new text of your targeted domain

#### rule-based identification system created from scratch

In [79]:
def docRB():
    f = open('test/doc4-RB.txt', 'w', encoding="utf8")
    doc = nlp(get_doc("Articles/txt/doc_04.txt"))
    annotate_doc(doc, f)
    f.close()
    
docRB()

Articles/txt/doc_04.txt || Text length:  22563


In [84]:
doctest2 = get_doc("test/doc4-RB.txt")
doctest2 = doctest2.replace("\n", " ")
doctest2 = doctest2.replace(" O ", " ")
doctest2 = doctest2.replace(" B ", " <B> ")
doctest2 = doctest2.replace(" I ", " <I> ")
doctest2 = doctest2.replace(" .", ".")
doctest2

test/doc4-RB.txt || Text length:  32227


'Linguistic Analysis of Toxic Behavior in an Online Video Game Haewoon Kwak † and Jeremy Blackburn ‡ †Qatar Computing Research Institute , Doha , Qatar hkwak@qf.org.qa ‡Telefonica Research , Barcelona , Spain jeremyb@tid.es Abstract.  In this paper we explore the linguistic components of toxic behavior by using crowdsourced data from over 590 thousand cases of accused toxic <B> players <I> in a popular match <B> - based <B> competition <B> game <B> , League of Legends.  We perform a series of linguistic analyses to gain a deeper understanding of the role <B> communication <B> plays <B> in the expression of toxic behavior.  We characterize linguistic behavior of toxic <B> players and <I> compare <I> it <I> with that of typical <B> players <I> in an online <B> competition game <I>.  We also find empirical support <B> describing how a player <B> transitions from typical to toxic behavior.  Our findings can be helpful to automatically detect and warn players <B> who may become toxic and th

#### Sequence tagger trained model from flair

In [85]:
from flair.data import Sentence

# load the model you trained
model = SequenceTagger.load('resources/taggers/example-pos/final-model.pt')

# create example sentence
sentence = Sentence(doctest)

# predict tags and print
model.predict(sentence)

doctest1 = sentence.to_tagged_string()
doctest1

2020-11-19 17:05:53,782 loading file resources/taggers/example-pos/final-model.pt


'Linguistic Analysis of Toxic Behavior in an Online Video Game <B> Haewoon Kwak † and Jeremy Blackburn ‡ † Qatar Computing Research Institute , Doha , Qatar hkwak @ qf.org.qa ‡ Telefonica Research , Barcelona , Spain jeremyb @ tid.es Abstract . In this paper we explore the linguistic components of toxic behavior by using crowdsourced data from over 590 thousand cases of accused toxic <B> players <I> in a popular match-based competition <B> game <B> , League of Legends . We perform a series of linguistic analyses to gain a deeper understanding of the role <B> communication plays <B> in the expression of toxic behavior . We characterize linguistic behavior of toxic <B> players <I> and compare it with that of typical players <B> in an online <B> competition <I> game <I> . We also find empirical support describing how a player <B> transitions from typical to toxic behavior . Our findings can be helpful to automatically detect and warn players <B> who may become toxic and thus insulate pote